In [18]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json

In [2]:
#Importamos el url
url = "https://raw.githubusercontent.com/olherreragz/EAE3709-2025-1/refs/heads/main/T2_data/Data.csv"
df = pd.read_csv(url)

In [3]:
#Eliminamos las columnas
df.drop(columns=['Country Code', 'Series Code'], inplace=True)

In [4]:
# Transformamos de long a wide
df = df.pivot(index="Country Name", columns="Series Name", values="2023 [YR2023]")

In [5]:
# Dejamos una columna de índice para tener disponible la de Pais.
df.reset_index(inplace=True)

In [6]:
elim_list = """ - "Africa Eastern and Southern"  
- "Africa Western and Central"  
- "Arab World"  
- "Caribbean small states"  
- "Central Europe and the Baltics"  
- "Early-demographic dividend"  
- "East Asia & Pacific"  
- "East Asia & Pacific (IDA & IBRD countries)"  
- "East Asia & Pacific (excluding high income)"  
- "Euro area"  
- "Europe & Central Asia"  
- "Europe & Central Asia (IDA & IBRD countries)"  
- "Europe & Central Asia (excluding high income)"  
- "European Union"  
- "Fragile and conflict affected situations"  
- "Heavily indebted poor countries (HIPC)"  
- "High income"  
- "IBRD only"  
- "IDA & IBRD total"  
- "IDA blend"  
- "IDA only"  
- "IDA total"  
- "Late-demographic dividend"  
- "Latin America & Caribbean"  
- "Latin America & Caribbean (excluding high income)"  
- "Latin America & the Caribbean (IDA & IBRD countries)"  
- "Least developed countries: UN classification"  
- "Low & middle income"  
- "Low income"  
- "Lower middle income"  
- "Middle East & North Africa"  
- "Middle East & North Africa (IDA & IBRD countries)"  
- "Middle East & North Africa (excluding high income)"  
- "Middle income"  
- "North America"  
- "Not classified"  
- "OECD members"  
- "Other small states"  
- "Pacific island small states"  
- "Post-demographic dividend"  
- "Pre-demographic dividend"  
- "Small states"  
- "South Asia"  
- "South Asia (IDA & IBRD)"  
- "Sub-Saharan Africa"  
- "Sub-Saharan Africa (IDA & IBRD countries)"  
- "Sub-Saharan Africa (excluding high income)"  
- "Upper middle income"  
- "World" """


In [7]:
# for entry in elim_list: Primero separa a cada país eliminando \n
# Luego, remueve '-' y despues las """", solo si hay algo (if entry).

elim_list = [entry.strip('- ').strip('"') for entry in elim_list.split("\n") if entry]

In [8]:
# eliminamos las columnas
df = df[~df["Country Name"].isin(elim_list)]

In [9]:
df = df.replace("..", np.nan)

In [10]:
df[df.columns[1:]] = df[df.columns[1:]].apply(pd.to_numeric, errors="coerce")

In [11]:
df.drop(columns="International tourism; receipts (% of total exports)", inplace=True)

In [12]:
# Load GeoJSON file containing country boundaries
url = "https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson"
world = gpd.read_file(url)

# Print column names to see what's available
print("Available columns:", world.columns.tolist())

# Create an empty dictionary for neighboring countries
neighbour_dict = {}

# Find neighbors by checking shared borders
for index, country in world.iterrows():
    # Use 'name' instead of 'ADMIN' as it's the correct column name in this dataset
    country_name = country['name']  # Changed from 'ADMIN' to 'name'
    neighbours = world[world.touches(country.geometry)]['name'].tolist()  # Changed here too
    neighbour_dict[country_name] = neighbours

# Print the dictionary of neighboring countries
print(neighbour_dict)

Available columns: ['name', 'ISO3166-1-Alpha-3', 'ISO3166-1-Alpha-2', 'geometry']
{'Indonesia': ['Malaysia', 'East Timor', 'Papua New Guinea'], 'Malaysia': ['Indonesia', 'Thailand', 'Brunei'], 'Chile': ['Bolivia', 'Peru', 'Argentina', 'Southern Patagonian Ice Field'], 'Bolivia': ['Chile', 'Peru', 'Argentina', 'Brazil', 'Paraguay'], 'Peru': ['Chile', 'Bolivia', 'Brazil', 'Ecuador', 'Colombia'], 'Argentina': ['Chile', 'Bolivia', 'Brazil', 'Uruguay', 'Paraguay', 'Brazilian Island', 'Southern Patagonian Ice Field'], 'Dhekelia Sovereign Base Area': ['Cyprus', 'Northern Cyprus', 'Cyprus No Mans Area'], 'Cyprus': ['Dhekelia Sovereign Base Area', 'Cyprus No Mans Area', 'Akrotiri Sovereign Base Area'], 'India': ['China', 'Bhutan', 'Pakistan', 'Myanmar', 'Bangladesh', 'Nepal', 'Siachen Glacier'], 'China': ['India', 'North Korea', 'Bhutan', 'Kazakhstan', 'Tajikistan', 'Mongolia', 'Russia', 'Vietnam', 'Laos', 'Kyrgyzstan', 'Pakistan', 'Myanmar', 'Afghanistan', 'Nepal', 'Hong Kong S.A.R.', 'Siachen

In [13]:
neighbour_dict.get("France")

['Suriname',
 'Brazil',
 'Germany',
 'Luxembourg',
 'Belgium',
 'Spain',
 'Italy',
 'Switzerland',
 'Monaco',
 'Andorra']

In [14]:
neighbour_dict.get("Suriname")

['France', 'Guyana', 'Brazil']

In [15]:
neighbour_dict["France"].remove("Suriname")

In [16]:
neighbour_dict["Suriname"].remove("France")

In [20]:
with open("country_neighbours.json", "w") as f:
    json.dump(neighbour_dict, f, indent=4)